# 自动语音识别 (ASR, Automatic speech recognition)
自动语音识别（ASR）是将语音信号转换为文本的技术，它将音频输入序列映射到文本输出。

像Siri和Alexa这样的虚拟助手每天都使用ASR模型来帮助用户，还有许多其他有用的面向用户的应用程序，例如会议期间的实时字幕和笔记。

本指南将向你展示如何：

1. **微调Wav 2 Vec 2**：在[MInDS-14](https://huggingface.co/datasets/PolyAI/minds14)数据集上微调[Wav 2 Vec 2](https://huggingface.co/facebook/wav2vec2-base)模型，将音频转录为文本。
2. **使用微调模型进行推理**：利用微调后的模型进行语音识别推理。

要查看与此任务兼容的所有体系结构和 checkpoints，我们建议你访问[任务页面](https://huggingface.co/tasks/automatic-speech-recognition)进行查看。

在开始之前，请确保你已经安装了所有必要的库：

In [ ]:
pip install transformers datasets evaluate jiwer

我们鼓励你登录你的Hugging Face账户，这样你就可以上传并与社区分享你的模型。当提示时，输入你的令牌进行登录：

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## 加载 MInDS-14 数据集
首先，从🤗 Datasets库中加载 [MInDS-14](https://huggingface.co/datasets/PolyAI/minds14) 数据集的一个较小子集。这将给你一个机会进行实验，确保一切正常工作，然后再花更多时间在完整数据集上进行训练。

In [ ]:
from datasets import load_dataset, Audio

minds = load_dataset("PolyAI/minds14", name="en-US", split="train[:100]")

使用 `Dataset.train_test_split` 方法将数据集的训练部分划分为训练集和测试集：

In [ ]:
minds = minds.train_test_split(test_size=0.2)

然后，查看数据集的内容：

In [ ]:
minds

虽然数据集包含许多有用信息，如lang_id和english_transcription，但本指南中你将专注于音频和转录文本。使用[remove_columns](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.remove_columns)方法移除其他列：

In [ ]:
minds = minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

数据集中有两个主要字段：

1. **audio**：一个1维数组，包含语音信号，必须调用以加载和重采样音频文件。
2. **transcription**：目标文本。

## 预处理数据

下一步是加载一个 Wav2Vec2 预训练的处理器来处理音频信号：

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

MInDS-14数据集的采样率为8000kHz（你可以在其数据集的 Readme 中找到这一信息），这意味着你需要将数据集重采样到16000kHz，以便使用预训练的Wav2Vec2模型：

In [ ]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

如上转录文本所示，文本包含大小写混合的字符。

**Wav2Vec2分词器只在大写字符上进行了训练，因此你需要确保文本中的内容都能够与分词器的词汇表相匹配：**

In [ ]:
def uppercase(example):
    return {"transcription": example["transcription"].upper()}


minds = minds.map(uppercase)

现在，创建一个预处理函数，该函数将：

1. 调用音频列以加载和重采样音频文件。
2. 从音频文件中提取输入值，并使用处理器对转录列进行分词。

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcription"])
    batch["input_length"] = len(batch["input_values"][0])
    return batch

要将对整个数据集应用预处理函数，使用🤗 Datasets的[map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map)函数。**你可以通过增加`num_proc`参数中的进程数来加速map操作。**使用[remove_columns](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.remove_columns)方法移除不需要的列：

In [ ]:
encoded_minds = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

🤗 Transformers库中没有为自动语音识别（ASR）提供现成的数据整理器，因此你需要调整[DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding)来创建一个示例批次。

它还将动态地填充你的文本和标签，使其与批次中最长元素的长度一致（而不是整个数据集的长度），从而使它们具有统一的长度。虽然在分词器函数中通过设置`padding=True`来填充文本是可能的，但使用动态填充会更加更高效。

与其他数据整理器不同，这个特定的数据整理器需要对`input_values`和`labels`应用不同的填充方法：

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    processor: AutoProcessor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 将输入和标签分开，因为它们必须具有不同的长度并且需要不同的填充方法。
        input_features = [{"input_values": feature["input_values"][0]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")

        # 用 -100 (表示无声音) 替换填充，以正确忽略损失。
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

现在，实例化你的`DataCollatorForCTCWithPadding`：

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding="longest")

## 评估模型

在训练过程中包含一个评估指标，通常有助于评估模型的性能。

你可以使用🤗 Evaluate库快速加载一个评估方法。对于这个任务，加载[词错误率（WER）](https://huggingface.co/spaces/evaluate-metric/wer)指标（查看[🤗 Evaluate 快速教程](https://huggingface.co/docs/evaluate/a_quick_tour)，了解更多关于如何加载和计算指标的信息）：

In [ ]:
import evaluate

wer = evaluate.load("wer")

然后，创建一个函数，将你的预测和标签传递给[compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute)函数来计算 WER：

In [ ]:
import numpy as np


def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

你的`compute_metrics`函数现在已经准备好了，在设置训练时你将再次使用它。

## 训练模型

如果你不熟悉如何使用Trainer进行模型微调，可以查看[这里](../tutorials/5_fine_tune_pretrained_model.ipynb)的基本教程！

现在你已经准备好开始训练模型了！使用[AutoModelForCTC](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForCTC)加载 `Wav2Vec2`。通过`ctc_loss_reduction`参数指定要应用的缩减方法。通常使用平均值比默认的求和更好：

In [ ]:
from transformers import AutoModelForCTC, TrainingArguments, Trainer

model = AutoModelForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

**`ctc_loss_reduction`参数说明**

`ctc_loss_reduction`参数并不直接指定要应用的损失函数，而是用于确定在计算连接时序分类（CTC）损失时如何处理多个时间步的损失值。具体来说，它决定了是将这些损失值求和（sum）还是取平均值（mean）。默认情况下，CTC损失是求和的，但有时使用平均值可以更好地平衡不同长度的输入序列对总损失的贡献。

- **sum**：将所有时间步的损失值相加。
- **mean**：将所有时间步的损失值取平均。

通过设置`ctc_loss_reduction`参数，你可以选择更适合你任务的处理方式。

至此，只剩下三个步骤：

1. 在[TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)中定义你的训练超参数。唯一必需的参数是`output_dir`，它指定了保存模型的位置。通过设置`push_to_hub=True`，你将把这个模型推送到Hub（你需要登录Hugging Face才能上传模型）。在每个 epoch 结束时，[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)将评估 WER 并保存训练检查点。
2. 将训练参数传递给[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)，同时包括模型、数据集、分词器、数据整理器和compute_metrics函数。
3. 调用[train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train)方法来开始微调你的模型。

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_asr_mind_model",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    processing_class=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

训练完成后，使用`push_to_hub()`方法将你的模型分享到Hub，这样每个人都可以使用你的模型：

In [ ]:
trainer.push_to_hub()

如果你想了解更多关于如何微调自动语音识别（ASR）模型的详细示例，可以查看以下博客文章：
- [英文ASR微调示例](https://huggingface.co/blog/fine-tune-wav2vec2-english)
- [多语言ASR微调示例](https://huggingface.co/blog/fine-tune-xlsr-wav2vec2)

## 模型推理

当你已经微调好了一个模型，就可以用它来进行推理了！

加载你想要进行推理的音频文件。如果需要重新采样的话，记得设置音频文件的采样率与模型的采样率一致！

In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", "en-US", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate
audio_file = dataset[0]["audio"]["path"]

尝试使用你的微调模型进行推理的最简单方法是使用`pipeline()`。实例化一个用于自动语音识别的pipeline，并将你的音频文件传递给它：

In [ ]:
from transformers import pipeline

transcriber = pipeline("automatic-speech-recognition", model="stevhliu/my_awesome_asr_minds_model")
transcriber(audio_file)

转录结果应该还可以更好！可以尝试在更多示例上微调你的模型，以获得更佳的结果！

你也可以手动复现`pipeline`的结果：

加载一个处理器来预处理音频文件和转录文本，并将输入转换为PyTorch张量：

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("stevhliu/my_awesome_asr_mind_model")
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

将你的输入传递给模型，并获取输出日志，查看概率分布（logits）：

In [ ]:
from transformers import AutoModelForCTC

model = AutoModelForCTC.from_pretrained("stevhliu/my_awesome_asr_mind_model")
with torch.no_grad():
    logits = model(**inputs).logits

获取具有最高概率的预测输入ID，并使用处理器将这些预测输入ID解码回文本：

In [ ]:
import torch

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
transcription